In [2]:
import tensorflow as tf
print(tf.__version__)

2.17.0


# 多输入通道和多输出通道

前面两节里我们用到的输入和输出都是二维数组，但真实数据的维度经常更高。例如，彩色图像在高和宽2个维度外还有RGB（红、绿、蓝）3个颜色通道。假设彩色图像的高和宽分别是$h$和$w$（像素），那么它可以表示为一个$3\times h\times w$的多维数组。我们将大小为3的这一维称为通道（channel）维。本节我们将介绍含多个输入通道或多个输出通道的卷积核。

In [3]:
def corr2d(X, K):
    """单通道二维卷积计算"""
    # 获取卷积核的高度和宽度
    h, w = K.shape
    # 如果输入X是一维的，将其重塑为二维
    if len(X.shape) <= 1:
        X = tf.reshape(X, (X.shape[0],1))
    
    # 创建输出变量Y，大小为 (X的高度-卷积核高度+1) x (X的宽度-卷积核宽度+1)
    # 初始化二维卷积操作结果形状
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    
    # 执行二维互相关运算
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 对应位置的元素相乘并求和，然后转换为float32类型
            # 计算X和K的对应元素相乘
            prod = X[i:i+h, j:j+w] * K
            # 对乘积进行求和
            sum_result = tf.reduce_sum(prod)
            # 将结果转换为float32类型
            casted_result = tf.cast(sum_result, dtype=tf.float32)
            # 将结果赋值给Y[i,j]
            Y[i,j].assign(casted_result)
    
    return Y

## 5.3.1 muti-channels in

当输入数据含多个通道时，我们需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算。假设输入数据的通道数为$c_i$，那么卷积核的输入通道数同样为$c_i$。设卷积核窗口形状为$k_h\times k_w$。当$c_i=1$时，我们知道卷积核只包含一个形状为$k_h\times k_w$的二维数组。当$c_i > 1$时，我们将会为每个输入通道各分配一个形状为$k_h\times k_w$的核数组。把这$c_i$个数组在输入通道维上连结，即得到一个形状为$c_i\times k_h\times k_w$的卷积核。由于输入和卷积核各有$c_i$个通道，我们可以在各个通道上对输入的二维数组和卷积核的二维核数组做互相关运算，再将这$c_i$个互相关运算的二维输出按通道相加，得到一个二维数组。这就是含多个通道的输入数据与多输入通道的卷积核做二维互相关运算的输出。

图5.4展示了含2个输入通道的二维互相关计算的例子。在每个通道上，二维输入数组与二维核数组做互相关运算，再按通道相加即得到输出。图5.4中阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$。

![含2个输入通道的互相关计算](../img/conv-multi-in.svg)


接下来我们实现含多个输入通道的互相关运算。我们只需要对每个通道做互相关运算，然后进行累加。

In [4]:
def corr2d_multi_in(X, K):
    # 创建一个列表来存储每个通道的互相关结果
    channel_results = []
    
    # 对每个通道进行互相关运算
    for i in range(X.shape[0]):
        channel_result = corr2d(X[i], K[i])
        channel_results.append(channel_result)
    
    # 在通道维度上对结果进行求和
    return tf.reduce_sum(channel_results, axis=0)

X = tf.constant([
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],  # 第一个通道
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]]   # 第二个通道
])  # X 的形状为 (2, 3, 3)，表示 2 个通道，每个通道是 3x3 的矩阵

K = tf.constant([
    [[0, 1], [2, 3]],  # 第一个通道的卷积核
    [[1, 2], [3, 4]]   # 第二个通道的卷积核
])  # K 的形状为 (2, 2, 2)，表示 2 个通道，每个通道是 2x2 的卷积核

result = corr2d_multi_in(X, K)
# print(result.shape)

## 5.3.2 multi-channels out

当输入通道有多个时，因为我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是为1。设卷积核输入通道数和输出通道数分别为$c_i$和$c_o$，高和宽分别为$k_h$和$k_w$。如果希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为$c_i\times k_h\times k_w$的核数组。将它们在输出通道维上连结，卷积核的形状即$c_o\times c_i\times k_h\times k_w$。在做互相关运算时，每个输出通道上的结果由卷积核在该输出通道上的核数组与整个输入数组计算而来。

下面我们实现一个互相关运算函数来计算多个通道的输出。

In [5]:
def corr2d_multi_in_out(X, K):
    """多输出通道二维卷积计算"""
    #多通道的结果
    results = []
    # 遍历每个输出通道的卷积核
    for k in K:
        # 对当前卷积核执行多输入通道的互相关运算
        result = corr2d_multi_in(X, k)
        # 将结果添加到列表中
        results.append(result)
    # 将所有结果在新的轴上堆叠，形成多输出通道的结果
    return tf.stack(results, axis=0)

我们将核数组`K`同`K+1`（`K`中每个元素加一）和`K+2`连结在一起来构造一个输出通道数为3的卷积核。

In [6]:
K = tf.stack([K, K+1, K+2],axis=0)
K.shape

TensorShape([3, 2, 2, 2])

下面我们对输入数组`X`与核数组`K`做互相关运算。此时的输出含有3个通道。其中第一个通道的结果与之前输入数组`X`与多输入通道、单输出通道核的计算结果一致。

## debugging

In [7]:
result = corr2d_multi_in_out(X, K)
result.shape

TensorShape([3, 2, 2])

## 5.3.3 1x1 convolution

最后我们讨论卷积窗口形状为$1\times 1$（$k_h=k_w=1$）的多通道卷积层。我们通常称之为$1\times 1$卷积层，并将其中的卷积运算称为$1\times 1$卷积。因为使用了最小窗口，$1\times 1$卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。实际上，$1\times 1$卷积的主要计算发生在通道维上。图5.5展示了使用输入通道数为3、输出通道数为2的$1\times 1$卷积核的互相关计算。值得注意的是，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。

![使用输入通道数为3、输出通道数为2的$1\times 1$卷积核的互相关计算。输入和输出具有相同的高和宽](../img/conv-1x1.svg)

下面我们使用全连接层中的矩阵乘法来实现$1\times 1$卷积。这里需要在矩阵乘法运算前后对数据形状做一些调整。

In [11]:
def corr2d_multi_in_out_1x1(X, K):
    """
    多输入通道、多输出通道的1x1卷积
    """
    # 输入通道数、高、宽
    c_i, h, w = X.shape
    # 输出通道数
    c_o = K.shape[0]
    # 将输入X重塑为(输入通道数, 高*宽)的形状，将二维的单通道数据展平为一维
    X = tf.reshape(X,(c_i, h * w))
    # 将卷积核K重塑为(输出通道数, 输入通道数)的形状
    K = tf.reshape(K,(c_o, c_i))
    # 执行矩阵乘法，相当于1x1卷积操作
    Y = tf.matmul(K, X)
    print(f"Y.shape: {Y.shape}, K.shape: {K.shape}, X.shape: {X.shape}")
    # 将结果重塑回原始的空间维度，得到(输出通道数, 高, 宽)的形状
    return tf.reshape(Y, (c_o, h, w))

![](../img/conv-1x1-draft.png)
图片内容是上述案例的思考过程

In [20]:
X = tf.random.uniform((3,3,3))
K = tf.random.uniform((2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(Y1.shape)
print(Y2.shape)


tf.norm(Y1-Y2) < 1e-6

Y.shape: (2, 9), K.shape: (2, 3), X.shape: (3, 9)
(2, 3, 3)
(2, 3, 3)


<tf.Tensor: shape=(), dtype=bool, numpy=True>

在之后的模型里我们将会看到$1\times 1$卷积层被当作保持高和宽维度形状不变的全连接层使用。于是，我们可以通过调整网络层之间的通道数来控制模型复杂度。


## 小结

* 使用多通道可以拓展卷积层的模型参数。
* 假设将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。
* $1\times 1$卷积层通常用来调整网络层之间的通道数，并控制模型复杂度。